### Sequential Bayesian learning visualization
Showing the prior, point/dataset likelihood and posterior distributions for points in a simple linear sequential bayesian learning model.

Author: Teun Mathijssen

Implementing Bishop p153-154

In [ ]:
%pylab inline

from scipy.stats import multivariate_normal as mvg

In [ ]:
# Simulation setup
N_iter = 100

resolution_plot = 61
plot_ll_dataset=True

a = -0.3, 0.5
alpha = 2
beta = 25

In [ ]:
def f(x, a):
    """Linear function."""
    return a[0] + a[1]*x

def gen_t(x, a, beta):
    """Generate sample with noise."""
    return f(x, a) + np.random.normal(0, 1/np.sqrt(beta))

def gen_Phi(x):
    """Generate design matrix."""
    return np.vstack((np.ones(x.size), x)).T

def uvg(x, mu, sigma):
    """Univariate Gaussian PDF."""
    return 1/(sigma*np.sqrt(2*np.pi))*np.exp(-((x-mu)/sigma)**2/2)

In [ ]:
# Grid variables for plotting.
x1 = x2 = np.linspace(-1, 1, resolution_plot)
X1, X2 = np.meshgrid(x1, x2)
X = np.dstack((X1, X2))
X_transpose = np.transpose(X, (2, 0, 1))

def plot_data(prior, a, m, S, x, t, ll_dataset=False):
    fig = plt.figure(figsize=(10, 5))
    
    # Likelihood plot.
    ax = plt.subplot(131)
    
    if x.size != 0:
        if ll_dataset:
            log_ll = np.zeros(X1.shape)
            for i in range(x.size):
                # Calculate log likelihood of dataset (add 1 to always make log >= 0).
                y = f(x[i], X_transpose)
                log_ll += np.log(uvg(t[i], y, beta**-1) + 1)

            # Normalize and exponentiate log likelihood values.
            ll = np.exp(log_ll/np.max(log_ll))
            
            ax.contourf(X1, X2, ll, resolution_plot, cmap="jet")
            ax.set_title("Dataset likelihood")
        else:
            # Calculate likelihood for last point.
            y = f(x[-1], X_transpose)
            ll = uvg(t[-1], y, beta**-1)
            
            ax.contourf(X1, X2, ll, resolution_plot, cmap="jet")
            ax.set_title("Point likelihood")
            
        ax.scatter(a[0], a[1], color="white", s=120, marker="+", linewidth=2)

    ax.set_aspect("equal")
    ax.set_xticks([-1, 0, 1])
    ax.set_yticks([-1, 0, 1])
    ax.set_xlabel("w1")
    ax.set_ylabel("w2", rotation=0)

    # Prior / posterior plot.
    ax = plt.subplot(132)
    ax.contourf(X1, X2, prior.pdf(X), resolution_plot, cmap="jet")
    ax.scatter(a[0], a[1], color="white", s=120, marker="+", linewidth=2)

    ax.set_aspect("equal")
    ax.set_xticks([-1, 0, 1])
    ax.set_yticks([-1, 0, 1])
    ax.set_xlabel("w1")
    ax.set_ylabel("w2", rotation=0)
    ax.set_title("Prior/posterior")
    
    # Samples plot.
    ax = plt.subplot(133)
    ax.scatter(x, t, color="black", s=40, facecolors="none")
    
    # Sample lines using the prior distribution over our weights.
    N_samples = 10
    samples = np.random.multivariate_normal(m, S, N_samples)
    for i in np.arange(N_samples):
        y = f(x1, samples[i])
        
        ax.plot(x1, y, color="red", linewidth=2, alpha=0.3)

    ax.set_aspect("equal")
    ax.set_xticks([-1, 0, 1])
    ax.set_yticks([-1, 0, 1])
    ax.set_xlabel("x")
    ax.set_ylabel("y", rotation=0)
    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_title("Data and weight samples")
    
    plt.subplots_adjust(bottom=0, top=1, left=0, right=1, hspace=0, wspace=0.4)
    plt.show()

In [ ]:
m_0 = np.zeros(2)
S_0 = (alpha**-1)*np.eye(2)
x = t = np.array([])

prior = mvg(m_0, S_0)
plot_data(prior, a, m_0, S_0, x, t)

for i in range(N_iter):
    cur_x = np.random.uniform(-1, 1)
    x = np.append(x, cur_x)
    t = np.append(t, gen_t(cur_x, a, beta))

    Phi = gen_Phi(x)

    S_N = np.linalg.inv(alpha*np.eye(2) + beta*(Phi.T @ Phi))
    m_N = beta*(S_N @ Phi.T @ t)
    
    posterior = mvg(m_N, S_N)
    
    plot_data(posterior, a, m_N, S_N, x, t, ll_dataset=plot_ll_dataset)